# Grading

In [1]:
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from pathlib import Path
import yaml
import shelve
import pandas as pd

In [2]:
def get_problem_names(assignment: int):
    d = yaml.safe_load(open("assignments.yml"))
    assignments = {a['id']: a for a in d}
    data = assignments[assignment]
    return data['problems']

In [3]:
ASSIGNMENT = 3

PROBLEMS = get_problem_names(ASSIGNMENT)

In [4]:
PROBLEMS

['combined-daily-time-series', 'dict-client', 'frankfurter']

In [5]:
setup_code = """
VERIFY_PROBLEM_GRADES = []
"""

save_results_code = """
import json

with open("/tmp/grades.json", "w") as f:
    json.dump(VERIFY_PROBLEM_GRADES, f)
"""

def get_code_cells(assignment):
    problems = get_problem_names(assignment)
    verify_cells = [f"%verify_problem {p}" for p in problems]
    return [setup_code] + verify_cells + [save_results_code]

def grade(assignment: int, path: Path):
    print("Grading", path)
    nb = nbformat.read(open(path), as_version=4)

    def add_code_cell(code):
        nb['cells'].append(nbformat.v4.new_code_cell(code))

    def prepend_front_matter(data):
        text = yaml.safe_dump(data)
        nb['cells'][0] = nbformat.v4.new_raw_cell(f"---\n{text}\n---")
        print("Added frontmatter")
              
    for code in get_code_cells(assignment):
        add_code_cell(code)
    
    p = ExecutePreprocessor(timeout=600, kernel_name='python3', allow_errors=True)
    nb, _ = p.preprocess(nb, {'metadata': {'path': '/tmp/'}})

    score = save_grade(path)    
    
    prepend_front_matter({
        "title": path.stem,
        "assignment": f"A{assignment:02d}",
        "author": path.stem,
        "score": score
    })

    outpath = Path("Submissions") / f"A{assignment:02d}" / path.name
    outpath.parent.mkdir(exist_ok=True, parents=True)
    with outpath.open("w") as f:
        nbformat.write(nb, f)
    
    print("saved", outpath)


def save_grade(path):
    path = Path(path)
    username = path.stem
    assignment = path.parent.name
    results = json.load(open("/tmp/grades.json"))

    correct = {p['problem'] for p in results if p['status'] == 'pass'}
    all_problems = {p['problem'] for p in results if p['status']}
    
    grade = {
        "username": username,
        "assignment": assignment,
        "score": len(correct),
        "max_score": len(all_problems),
        "results": results
    }
    key = f"{assignment}/{username}"
    
    with shelve.open("grades.db")  as db:
        db[key] = grade

    return len(correct)

def grade_assignments(assignment):
    paths = Path(f"archive/assignment-{assignment:02d}").glob("*.ipynb")
    paths = list(paths)
    
    for p in paths:
        grade(assignment, p)

    generate_index(assignment)

INDEX = """\
---
title: "Assignment {assignment:02d}"
listing:
  type: table
  contents:
    - "*.ipynb"
  fields:
   - title
   - assignment
   - file-modified
   - score
---
"""

def generate_index(assignment: int):
    path = Path("Submissions") / f"A{assignment:02d}" / "index.qmd"
    text = INDEX.format(assignment=assignment)
    path.write_text(text)

In [6]:
!sudo python manage.py collect -a 3

<Assignment#3 rohit_singh>
cp /home/jupyter-rohit_singh/assignments/assignment-03.ipynb archive/assignment-03/rohit_singh.ipynb
<Assignment#3 simran_banka>
cp /home/jupyter-simran_banka/assignments/assignment-03.ipynb archive/assignment-03/simran_banka.ipynb
<Assignment#3 tejeswar_ette>
cp /home/jupyter-tejeswar_ette/assignments/assignment-03.ipynb archive/assignment-03/tejeswar_ette.ipynb
<Assignment#3 chaitanya_datar>
cp /home/jupyter-chaitanya_datar/assignments/assignment-03.ipynb archive/assignment-03/chaitanya_datar.ipynb
<Assignment#3 ansh_paliwal>
cp /home/jupyter-ansh_paliwal/assignments/assignment-03.ipynb archive/assignment-03/ansh_paliwal.ipynb
<Assignment#3 rishita_goyanka>
cp /home/jupyter-rishita_goyanka/assignments/assignment-03.ipynb archive/assignment-03/rishita_goyanka.ipynb
<Assignment#3 turiganti_kumar>
cp /home/jupyter-turiganti_kumar/assignments/assignment-03.ipynb archive/assignment-03/turiganti_kumar.ipynb
<Assignment#3 gurupreet_mathadu>
cp /home/jupyter-gurupr

In [7]:
grade_assignments(ASSIGNMENT)

Grading archive/assignment-03/satyam_batra.ipynb
Added frontmatter
saved Submissions/A03/satyam_batra.ipynb
Grading archive/assignment-03/shagun_yadav.ipynb
Added frontmatter
saved Submissions/A03/shagun_yadav.ipynb
Grading archive/assignment-03/sahil_arora.ipynb
Added frontmatter
saved Submissions/A03/sahil_arora.ipynb
Grading archive/assignment-03/shubham_sagar.ipynb
Added frontmatter
saved Submissions/A03/shubham_sagar.ipynb
Grading archive/assignment-03/aman_vithalni.ipynb
Added frontmatter
saved Submissions/A03/aman_vithalni.ipynb
Grading archive/assignment-03/swapnil_agrawal.ipynb
Added frontmatter
saved Submissions/A03/swapnil_agrawal.ipynb
Grading archive/assignment-03/chaitanya_datar.ipynb
Added frontmatter
saved Submissions/A03/chaitanya_datar.ipynb
Grading archive/assignment-03/himakar.ipynb
Added frontmatter
saved Submissions/A03/himakar.ipynb
Grading archive/assignment-03/akshit_singhi.ipynb
Added frontmatter
saved Submissions/A03/akshit_singhi.ipynb
Grading archive/assign

In [8]:
!python manage.py report -a 3

assignment-03 satyam_batra 0 3
assignment-03 shagun_yadav 1 3
assignment-03 sahil_arora 1 3
assignment-03 shubham_sagar 0 3
assignment-03 aman_vithalni 1 3
assignment-03 swapnil_agrawal 0 3
assignment-03 chaitanya_datar 0 3
assignment-03 himakar 0 3
assignment-03 akshit_singhi 0 3
assignment-03 simran_banka 0 3
assignment-03 harish_padhi 1 3
assignment-03 prasad_ankulwar 2 3
assignment-03 rishi_brahma 0 3
assignment-03 gurupreet_mathadu 2 3
assignment-03 turiganti_kumar 1 3
assignment-03 rayala_sai 1 3
assignment-03 vinothkumar_v 0 3
assignment-03 rishita_goyanka 0 3
assignment-03 tejeswar_ette 1 3
assignment-03 vikrant 0 3
assignment-03 shuddhatma_jain 0 3
assignment-03 ansh_paliwal 1 3
assignment-03 rohit_singh 1 3


## Assignment 02

In [42]:
grade_assignments(2)

Grading archive/assignment-02/annapurna.ipynb
Added frontmatter
saved Submissions/A02/annapurna.ipynb
Grading archive/assignment-02/saleem.ipynb
Added frontmatter
saved Submissions/A02/saleem.ipynb
Grading archive/assignment-02/soumya.ipynb
Added frontmatter
saved Submissions/A02/soumya.ipynb
Grading archive/assignment-02/zoya.ipynb
Added frontmatter
saved Submissions/A02/zoya.ipynb
Grading archive/assignment-02/anand.ipynb
Added frontmatter
saved Submissions/A02/anand.ipynb
Grading archive/assignment-02/abhishek.ipynb
Added frontmatter
saved Submissions/A02/abhishek.ipynb
Grading archive/assignment-02/karan.ipynb
Added frontmatter
saved Submissions/A02/karan.ipynb
Grading archive/assignment-02/anish.ipynb
Added frontmatter
saved Submissions/A02/anish.ipynb
Grading archive/assignment-02/vipul.ipynb
Added frontmatter
saved Submissions/A02/vipul.ipynb
Grading archive/assignment-02/akshay.ipynb
Added frontmatter
saved Submissions/A02/akshay.ipynb
Grading archive/assignment-02/shivmani.ipy

In [28]:
!python manage.py report -a2

assignment-02 annapurna 10 10
assignment-02 saleem 10 10
assignment-02 soumya 2 10
assignment-02 zoya 9 10
assignment-02 anand 0 10
assignment-02 abhishek 10 10
assignment-02 karan 10 10
assignment-02 anish 10 10
assignment-02 vipul 10 10
assignment-02 akshay 10 10
assignment-02 shivmani 5 10
assignment-02 manik 10 10
assignment-02 bhuvan 9 10
assignment-02 bindhushree 9 10
assignment-02 adithya 8 10
assignment-02 lokesh 5 10
assignment-02 sindhu 6 10
assignment-02 anushka 3 10
assignment-02 aravindhan 9 10
assignment-02 vinay 8 10


In [26]:
!python manage.py report -a2

assignment-02 annapurna 10 10
assignment-02 saleem 10 10
assignment-02 soumya 2 9
assignment-02 zoya 9 9
assignment-02 anand 0 9
assignment-02 abhishek 9 9
assignment-02 karan 10 10
assignment-02 anish 10 10
assignment-02 vipul 9 9
assignment-02 akshay 10 10
assignment-02 shivmani 5 9
assignment-02 manik 10 10
assignment-02 bhuvan 9 10
assignment-02 bindhushree 9 9
assignment-02 adithya 6 9
assignment-02 lokesh 0 9
assignment-02 sindhu 5 9
assignment-02 anushka 0 9
assignment-02 aravindhan 9 9
assignment-02 vinay 8 9


## Assignment 03

In [2]:
!python manage.py collect -a 3

<Assignment#3 soumya>
cp /home/jupyter-soumya/assignments/assignment-03.ipynb archive/assignment-03/soumya.ipynb
<Assignment#3 akshay>
cp /home/jupyter-akshay/assignments/assignment-03.ipynb archive/assignment-03/akshay.ipynb
<Assignment#3 bhuvan>
cp /home/jupyter-bhuvan/assignments/assignment-03.ipynb archive/assignment-03/bhuvan.ipynb
<Assignment#3 bindhushree>
cp /home/jupyter-bindhushree/assignments/assignment-03.ipynb archive/assignment-03/bindhushree.ipynb
<Assignment#3 karan>
cp /home/jupyter-karan/assignments/assignment-03.ipynb archive/assignment-03/karan.ipynb
<Assignment#3 zoya>
cp /home/jupyter-zoya/assignments/assignment-03.ipynb archive/assignment-03/zoya.ipynb
<Assignment#3 sindhu>
cp /home/jupyter-sindhu/assignments/assignment-03.ipynb archive/assignment-03/sindhu.ipynb
<Assignment#3 vinay>
cp /home/jupyter-vinay/assignments/assignment-03.ipynb archive/assignment-03/vinay.ipynb
<Assignment#3 anushka>
cp /home/jupyter-anushka/assignments/assignment-03.ipynb archive/assig

In [43]:
grade_assignments(3)

Grading archive/assignment-03/annapurna.ipynb
Added frontmatter
saved Submissions/A03/annapurna.ipynb
Grading archive/assignment-03/saleem.ipynb
Added frontmatter
saved Submissions/A03/saleem.ipynb
Grading archive/assignment-03/soumya.ipynb
Added frontmatter
saved Submissions/A03/soumya.ipynb
Grading archive/assignment-03/zoya.ipynb
Added frontmatter
saved Submissions/A03/zoya.ipynb
Grading archive/assignment-03/anand.ipynb
Added frontmatter
saved Submissions/A03/anand.ipynb
Grading archive/assignment-03/abhishek.ipynb
Added frontmatter
saved Submissions/A03/abhishek.ipynb
Grading archive/assignment-03/karan.ipynb
Added frontmatter
saved Submissions/A03/karan.ipynb
Grading archive/assignment-03/anish.ipynb
Added frontmatter
saved Submissions/A03/anish.ipynb
Grading archive/assignment-03/vipul.ipynb
Added frontmatter
saved Submissions/A03/vipul.ipynb
Grading archive/assignment-03/akshay.ipynb
Added frontmatter
saved Submissions/A03/akshay.ipynb
Grading archive/assignment-03/shivmani.ipy

In [22]:
!python manage.py report -a3

assignment-03 annapurna 5 5
assignment-03 saleem 5 5
assignment-03 soumya 3 5
assignment-03 zoya 4 5
assignment-03 anand 0 5
assignment-03 abhishek 0 5
assignment-03 karan 5 5
assignment-03 anish 5 5
assignment-03 vipul 5 5
assignment-03 akshay 5 5
assignment-03 shivmani 3 5
assignment-03 manik 5 5
assignment-03 bhuvan 5 5
assignment-03 bindhushree 5 5
assignment-03 adithya 5 5
assignment-03 lokesh 0 5
assignment-03 sindhu 5 5
assignment-03 anushka 0 5
assignment-03 aravindhan 5 5
assignment-03 vinay 5 5


## Assignment 04

In [2]:
!python manage.py collect -a 4

<Assignment#4 soumya>
cp /home/jupyter-soumya/assignments/assignment-04.ipynb archive/assignment-04/soumya.ipynb
<Assignment#4 akshay>
cp /home/jupyter-akshay/assignments/assignment-04.ipynb archive/assignment-04/akshay.ipynb
<Assignment#4 bhuvan>
cp /home/jupyter-bhuvan/assignments/assignment-04.ipynb archive/assignment-04/bhuvan.ipynb
<Assignment#4 bindhushree>
cp /home/jupyter-bindhushree/assignments/assignment-04.ipynb archive/assignment-04/bindhushree.ipynb
<Assignment#4 karan>
cp /home/jupyter-karan/assignments/assignment-04.ipynb archive/assignment-04/karan.ipynb
<Assignment#4 zoya>
cp /home/jupyter-zoya/assignments/assignment-04.ipynb archive/assignment-04/zoya.ipynb
<Assignment#4 sindhu>
cp /home/jupyter-sindhu/assignments/assignment-04.ipynb archive/assignment-04/sindhu.ipynb
<Assignment#4 vinay>
cp /home/jupyter-vinay/assignments/assignment-04.ipynb archive/assignment-04/vinay.ipynb
<Assignment#4 anushka>
cp /home/jupyter-anushka/assignments/assignment-04.ipynb archive/assig

In [10]:
grade_assignments(4)

Grading archive/assignment-04/annapurna.ipynb
Added frontmatter
saved Submissions/A04/annapurna.ipynb
Grading archive/assignment-04/saleem.ipynb
Added frontmatter
saved Submissions/A04/saleem.ipynb
Grading archive/assignment-04/soumya.ipynb
Added frontmatter
saved Submissions/A04/soumya.ipynb
Grading archive/assignment-04/zoya.ipynb
Added frontmatter
saved Submissions/A04/zoya.ipynb
Grading archive/assignment-04/anand.ipynb
Added frontmatter
saved Submissions/A04/anand.ipynb
Grading archive/assignment-04/abhishek.ipynb
Added frontmatter
saved Submissions/A04/abhishek.ipynb
Grading archive/assignment-04/karan.ipynb
Added frontmatter
saved Submissions/A04/karan.ipynb
Grading archive/assignment-04/anish.ipynb
Added frontmatter
saved Submissions/A04/anish.ipynb
Grading archive/assignment-04/vipul.ipynb
Added frontmatter
saved Submissions/A04/vipul.ipynb
Grading archive/assignment-04/akshay.ipynb
Added frontmatter
saved Submissions/A04/akshay.ipynb
Grading archive/assignment-04/shivmani.ipy

In [3]:
for i in range(1, 5):
    generate_index(i)

NameError: name 'generate_index' is not defined

In [11]:
!python manage.py report -a4

assignment-04 annapurna 5 5
assignment-04 saleem 5 5
assignment-04 soumya 5 5
assignment-04 zoya 5 5
assignment-04 anand 0 5
assignment-04 abhishek 4 5
assignment-04 karan 4 5
assignment-04 anish 5 5
assignment-04 vipul 4 5
assignment-04 akshay 4 5
assignment-04 shivmani 1 5
assignment-04 manik 5 5
assignment-04 bhuvan 5 5
assignment-04 bindhushree 4 5
assignment-04 adithya 4 5
assignment-04 lokesh 0 5
assignment-04 sindhu 4 5
assignment-04 anushka 0 5
assignment-04 aravindhan 5 5
assignment-04 vinay 5 5


## Assignment 05

In [7]:
# !python manage.py collect -a 5

In [8]:
grade_assignments(5)

Grading archive/assignment-05/annapurna.ipynb
Added frontmatter
saved Submissions/A05/annapurna.ipynb
Grading archive/assignment-05/saleem.ipynb
Added frontmatter
saved Submissions/A05/saleem.ipynb
Grading archive/assignment-05/soumya.ipynb
Added frontmatter
saved Submissions/A05/soumya.ipynb
Grading archive/assignment-05/zoya.ipynb
Added frontmatter
saved Submissions/A05/zoya.ipynb
Grading archive/assignment-05/anand.ipynb
Added frontmatter
saved Submissions/A05/anand.ipynb
Grading archive/assignment-05/abhishek.ipynb
Added frontmatter
saved Submissions/A05/abhishek.ipynb
Grading archive/assignment-05/karan.ipynb
Added frontmatter
saved Submissions/A05/karan.ipynb
Grading archive/assignment-05/anish.ipynb
Added frontmatter
saved Submissions/A05/anish.ipynb
Grading archive/assignment-05/vipul.ipynb
Added frontmatter
saved Submissions/A05/vipul.ipynb
Grading archive/assignment-05/akshay.ipynb
Added frontmatter
saved Submissions/A05/akshay.ipynb
Grading archive/assignment-05/shivmani.ipy

## Assignment 06

In [20]:
# !python manage.py collect -a 6

In [28]:
grade_assignments(6)

Grading archive/assignment-06/annapurna.ipynb
Added frontmatter
saved Submissions/A06/annapurna.ipynb
Grading archive/assignment-06/saleem.ipynb
Added frontmatter
saved Submissions/A06/saleem.ipynb
Grading archive/assignment-06/soumya.ipynb
Added frontmatter
saved Submissions/A06/soumya.ipynb
Grading archive/assignment-06/zoya.ipynb
Added frontmatter
saved Submissions/A06/zoya.ipynb
Grading archive/assignment-06/anand.ipynb
Added frontmatter
saved Submissions/A06/anand.ipynb
Grading archive/assignment-06/abhishek.ipynb
Added frontmatter
saved Submissions/A06/abhishek.ipynb
Grading archive/assignment-06/karan.ipynb
Added frontmatter
saved Submissions/A06/karan.ipynb
Grading archive/assignment-06/anish.ipynb
Added frontmatter
saved Submissions/A06/anish.ipynb
Grading archive/assignment-06/vipul.ipynb
Added frontmatter
saved Submissions/A06/vipul.ipynb
Grading archive/assignment-06/akshay.ipynb
Added frontmatter
saved Submissions/A06/akshay.ipynb
Grading archive/assignment-06/shivmani.ipy

## Assignment 07

In [6]:
!python manage.py collect -a 7

<Assignment#7 soumya>
cp /home/jupyter-soumya/assignments/assignment-07.ipynb archive/assignment-07/soumya.ipynb
<Assignment#7 akshay>
cp /home/jupyter-akshay/assignments/assignment-07.ipynb archive/assignment-07/akshay.ipynb
<Assignment#7 bhuvan>
cp /home/jupyter-bhuvan/assignments/assignment-07.ipynb archive/assignment-07/bhuvan.ipynb
<Assignment#7 bindhushree>
cp /home/jupyter-bindhushree/assignments/assignment-07.ipynb archive/assignment-07/bindhushree.ipynb
<Assignment#7 karan>
cp /home/jupyter-karan/assignments/assignment-07.ipynb archive/assignment-07/karan.ipynb
<Assignment#7 zoya>
cp /home/jupyter-zoya/assignments/assignment-07.ipynb archive/assignment-07/zoya.ipynb
<Assignment#7 sindhu>
cp /home/jupyter-sindhu/assignments/assignment-07.ipynb archive/assignment-07/sindhu.ipynb
<Assignment#7 vinay>
cp /home/jupyter-vinay/assignments/assignment-07.ipynb archive/assignment-07/vinay.ipynb
<Assignment#7 anushka>
cp /home/jupyter-anushka/assignments/assignment-07.ipynb archive/assig

In [7]:
grade_assignments(7)

Grading archive/assignment-07/annapurna.ipynb
Added frontmatter
saved Submissions/A07/annapurna.ipynb
Grading archive/assignment-07/saleem.ipynb
Added frontmatter
saved Submissions/A07/saleem.ipynb
Grading archive/assignment-07/soumya.ipynb
Added frontmatter
saved Submissions/A07/soumya.ipynb
Grading archive/assignment-07/zoya.ipynb
Added frontmatter
saved Submissions/A07/zoya.ipynb
Grading archive/assignment-07/anand.ipynb
Added frontmatter
saved Submissions/A07/anand.ipynb
Grading archive/assignment-07/abhishek.ipynb
Added frontmatter
saved Submissions/A07/abhishek.ipynb
Grading archive/assignment-07/karan.ipynb
Added frontmatter
saved Submissions/A07/karan.ipynb
Grading archive/assignment-07/anish.ipynb
Added frontmatter
saved Submissions/A07/anish.ipynb
Grading archive/assignment-07/vipul.ipynb
Added frontmatter
saved Submissions/A07/vipul.ipynb
Grading archive/assignment-07/akshay.ipynb
Added frontmatter
saved Submissions/A07/akshay.ipynb
Grading archive/assignment-07/shivmani.ipy

### Assignment 8

In [8]:
!python manage.py collect -a 8

<Assignment#8 soumya>
cp /home/jupyter-soumya/assignments/assignment-08.ipynb archive/assignment-08/soumya.ipynb
<Assignment#8 akshay>
cp /home/jupyter-akshay/assignments/assignment-08.ipynb archive/assignment-08/akshay.ipynb
<Assignment#8 bhuvan>
cp /home/jupyter-bhuvan/assignments/assignment-08.ipynb archive/assignment-08/bhuvan.ipynb
<Assignment#8 bindhushree>
cp /home/jupyter-bindhushree/assignments/assignment-08.ipynb archive/assignment-08/bindhushree.ipynb
<Assignment#8 karan>
cp /home/jupyter-karan/assignments/assignment-08.ipynb archive/assignment-08/karan.ipynb
<Assignment#8 zoya>
cp /home/jupyter-zoya/assignments/assignment-08.ipynb archive/assignment-08/zoya.ipynb
<Assignment#8 sindhu>
cp /home/jupyter-sindhu/assignments/assignment-08.ipynb archive/assignment-08/sindhu.ipynb
<Assignment#8 vinay>
cp /home/jupyter-vinay/assignments/assignment-08.ipynb archive/assignment-08/vinay.ipynb
<Assignment#8 anushka>
cp /home/jupyter-anushka/assignments/assignment-08.ipynb archive/assig

In [9]:
grade_assignments(8)

Grading archive/assignment-08/annapurna.ipynb
Added frontmatter
saved Submissions/A08/annapurna.ipynb
Grading archive/assignment-08/saleem.ipynb


dig: couldn't get address for '@dns.toys': not found


Added frontmatter
saved Submissions/A08/saleem.ipynb
Grading archive/assignment-08/soumya.ipynb
Added frontmatter
saved Submissions/A08/soumya.ipynb
Grading archive/assignment-08/zoya.ipynb
Added frontmatter
saved Submissions/A08/zoya.ipynb
Grading archive/assignment-08/anand.ipynb
Added frontmatter
saved Submissions/A08/anand.ipynb
Grading archive/assignment-08/abhishek.ipynb
Added frontmatter
saved Submissions/A08/abhishek.ipynb
Grading archive/assignment-08/karan.ipynb
Added frontmatter
saved Submissions/A08/karan.ipynb
Grading archive/assignment-08/anish.ipynb


dig: couldn't get address for '': not found
dig: couldn't get address for '': not found


Added frontmatter
saved Submissions/A08/anish.ipynb
Grading archive/assignment-08/vipul.ipynb
Added frontmatter
saved Submissions/A08/vipul.ipynb
Grading archive/assignment-08/akshay.ipynb
Added frontmatter
saved Submissions/A08/akshay.ipynb
Grading archive/assignment-08/shivmani.ipynb
Added frontmatter
saved Submissions/A08/shivmani.ipynb
Grading archive/assignment-08/manik.ipynb
Added frontmatter
saved Submissions/A08/manik.ipynb
Grading archive/assignment-08/bhuvan.ipynb
Added frontmatter
saved Submissions/A08/bhuvan.ipynb
Grading archive/assignment-08/bindhushree.ipynb
Added frontmatter
saved Submissions/A08/bindhushree.ipynb
Grading archive/assignment-08/adithya.ipynb
Added frontmatter
saved Submissions/A08/adithya.ipynb
Grading archive/assignment-08/lokesh.ipynb
Added frontmatter
saved Submissions/A08/lokesh.ipynb
Grading archive/assignment-08/sindhu.ipynb
Added frontmatter
saved Submissions/A08/sindhu.ipynb
Grading archive/assignment-08/anushka.ipynb
Added frontmatter
saved Subm

### Assignment 09

In [1]:
!python manage.py collect -a 9

<Assignment#9 soumya>
cp /home/jupyter-soumya/assignments/assignment-09.ipynb archive/assignment-09/soumya.ipynb
<Assignment#9 akshay>
cp /home/jupyter-akshay/assignments/assignment-09.ipynb archive/assignment-09/akshay.ipynb
<Assignment#9 bhuvan>
cp /home/jupyter-bhuvan/assignments/assignment-09.ipynb archive/assignment-09/bhuvan.ipynb
<Assignment#9 bindhushree>
cp /home/jupyter-bindhushree/assignments/assignment-09.ipynb archive/assignment-09/bindhushree.ipynb
<Assignment#9 karan>
cp /home/jupyter-karan/assignments/assignment-09.ipynb archive/assignment-09/karan.ipynb
<Assignment#9 zoya>
cp /home/jupyter-zoya/assignments/assignment-09.ipynb archive/assignment-09/zoya.ipynb
<Assignment#9 sindhu>
cp /home/jupyter-sindhu/assignments/assignment-09.ipynb archive/assignment-09/sindhu.ipynb
<Assignment#9 vinay>
cp /home/jupyter-vinay/assignments/assignment-09.ipynb archive/assignment-09/vinay.ipynb
<Assignment#9 anushka>
cp /home/jupyter-anushka/assignments/assignment-09.ipynb archive/assig

In [56]:
grade_assignments(9)

Grading archive/assignment-09/annapurna.ipynb
Added frontmatter
saved Submissions/A09/annapurna.ipynb
Grading archive/assignment-09/saleem.ipynb
Added frontmatter
saved Submissions/A09/saleem.ipynb
Grading archive/assignment-09/soumya.ipynb
Added frontmatter
saved Submissions/A09/soumya.ipynb
Grading archive/assignment-09/zoya.ipynb
Added frontmatter
saved Submissions/A09/zoya.ipynb
Grading archive/assignment-09/anand.ipynb
Added frontmatter
saved Submissions/A09/anand.ipynb
Grading archive/assignment-09/abhishek.ipynb
Added frontmatter
saved Submissions/A09/abhishek.ipynb
Grading archive/assignment-09/karan.ipynb
Added frontmatter
saved Submissions/A09/karan.ipynb
Grading archive/assignment-09/anish.ipynb
Added frontmatter
saved Submissions/A09/anish.ipynb
Grading archive/assignment-09/vipul.ipynb
Added frontmatter
saved Submissions/A09/vipul.ipynb
Grading archive/assignment-09/akshay.ipynb
Added frontmatter
saved Submissions/A09/akshay.ipynb
Grading archive/assignment-09/shivmani.ipy

## Assignment 10

In [1]:
!python manage.py collect -a 10

<Assignment#10 soumya>
cp /home/jupyter-soumya/assignments/assignment-10.ipynb archive/assignment-10/soumya.ipynb
<Assignment#10 akshay>
cp /home/jupyter-akshay/assignments/assignment-10.ipynb archive/assignment-10/akshay.ipynb
<Assignment#10 bhuvan>
cp /home/jupyter-bhuvan/assignments/assignment-10.ipynb archive/assignment-10/bhuvan.ipynb
<Assignment#10 bindhushree>
cp /home/jupyter-bindhushree/assignments/assignment-10.ipynb archive/assignment-10/bindhushree.ipynb
<Assignment#10 karan>
cp /home/jupyter-karan/assignments/assignment-10.ipynb archive/assignment-10/karan.ipynb
<Assignment#10 zoya>
cp /home/jupyter-zoya/assignments/assignment-10.ipynb archive/assignment-10/zoya.ipynb
<Assignment#10 sindhu>
cp /home/jupyter-sindhu/assignments/assignment-10.ipynb archive/assignment-10/sindhu.ipynb
<Assignment#10 vinay>
cp /home/jupyter-vinay/assignments/assignment-10.ipynb archive/assignment-10/vinay.ipynb
<Assignment#10 anushka>
cp /home/jupyter-anushka/assignments/assignment-10.ipynb arch

## Report Table

In [15]:
from tabulate import tabulate
import shelve
import pandas as pd

In [6]:
with shelve.open("grades.db")  as db:
    data = list(db.values())

In [7]:
assignments = {submission['assignment'] for submission in data}

In [8]:
assignment_numbers = sorted([int(a.replace("assignment-", "")) for a in assignments])
usernames = sorted({submission['username'] for submission in data})

usernames.remove("anand")

In [9]:
submissions = {}
for sub in data:
    assignment_num = int(sub['assignment'].replace("assignment-", ""))
    username = sub['username']
    submissions.setdefault(username, {}).setdefault(assignment_num, sub)

In [10]:
assignment_numbers

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
def get_row(username):
    return [username] + [submissions[username].get(a, {}).get('score', 0) for a in assignment_numbers]

In [12]:
submissions.keys()

dict_keys(['abhishek', 'annapurna', 'saleem', 'soumya', 'zoya', 'anand', 'karan', 'anish', 'vipul', 'akshay', 'shivmani', 'manik', 'bhuvan', 'bindhushree', 'adithya', 'lokesh', 'sindhu', 'anushka', 'aravindhan', 'vinay'])

In [13]:
usernames

['abhishek',
 'adithya',
 'akshay',
 'anish',
 'annapurna',
 'anushka',
 'aravindhan',
 'bhuvan',
 'bindhushree',
 'karan',
 'lokesh',
 'manik',
 'saleem',
 'shivmani',
 'sindhu',
 'soumya',
 'vinay',
 'vipul',
 'zoya']

In [14]:
report = [get_row(username) for username in usernames]
names = ['Name'] + [f"A{a:02d}" for a in assignment_numbers]
df = pd.DataFrame(report, columns=names)

df.set_index("Name", inplace=True)

NameError: name 'pd' is not defined

In [67]:
df['Total'] = df.sum(axis=1)

In [68]:
df.to_markdown('grading-report.qmd')

In [69]:
df

,A01,A02,A03,A04,A05,A06,A07,A08,A09,Total
Name,,,,,,,,,,
abhishek,5,10,0,4,0,0,0,0,1,20
adithya,5,8,5,4,5,5,2,3,3,40
akshay,5,10,5,4,4,4,3,2,2,39
anish,5,10,5,5,5,4,0,3,1,38
annapurna,5,10,5,5,2,0,0,0,1,28
anushka,4,3,0,0,0,0,0,0,1,8
aravindhan,5,9,5,5,3,4,4,3,2,40
bhuvan,5,9,5,5,5,5,2,3,1,40
bindhushree,5,9,5,4,5,5,4,3,3,43


## Assignment 01

In [43]:
paths = Path("archive/assignment-01").glob("*.ipynb")

for p in paths:
    grade(p)

Grading archive/assignment-01/annapurna.ipynb
Grading archive/assignment-01/saleem.ipynb
Grading archive/assignment-01/soumya.ipynb
Grading archive/assignment-01/zoya.ipynb
Grading archive/assignment-01/anand.ipynb
Grading archive/assignment-01/abhishek.ipynb
Grading archive/assignment-01/karan.ipynb
Grading archive/assignment-01/anish.ipynb
Grading archive/assignment-01/vipul.ipynb
Grading archive/assignment-01/akshay.ipynb
Grading archive/assignment-01/shivmani.ipynb
Grading archive/assignment-01/manik.ipynb
Grading archive/assignment-01/bhuvan.ipynb
Grading archive/assignment-01/bindhushree.ipynb
Grading archive/assignment-01/adithya.ipynb
Grading archive/assignment-01/lokesh.ipynb
Grading archive/assignment-01/sindhu.ipynb
Grading archive/assignment-01/anushka.ipynb
Grading archive/assignment-01/aravindhan.ipynb
Grading archive/assignment-01/vinay.ipynb


In [41]:
with shelve.open("grades.db")  as db:
    for x in db.values():
        print(x['assignment'], x['username'], x['score'], x['max_score'])

assignment-01 abhishek 5 5
assignment-01 annapurna 2 5
assignment-01 saleem 5 5
assignment-01 soumya 5 5
assignment-01 zoya 5 5
assignment-01 anand 0 5
assignment-01 karan 5 5
assignment-01 anish 5 5
assignment-01 vipul 5 5
assignment-01 akshay 5 5
assignment-01 shivmani 5 5
assignment-01 manik 5 5
assignment-01 bhuvan 5 5
assignment-01 bindhushree 5 5
assignment-01 adithya 5 5
assignment-01 lokesh 4 5
assignment-01 sindhu 5 5
assignment-01 anushka 4 6
assignment-01 aravindhan 5 5
assignment-01 vinay 5 5


In [48]:
with shelve.open("grades.db")  as db:
    print(db['assignment-01/abhishek'])

{'username': 'abhishek', 'assignment': 'assignment-01', 'score': 5, 'max_score': 5, 'results': [{'problem': 'mean', 'status': 'pass', 'log': ['✓ mean([1, 2, 3, 4, 5])', '✓ mean([5])', '🎉 Congratulations! You have successfully solved problem mean!!']}, {'problem': 'product', 'status': 'pass', 'log': ['✓ product([1, 2, 3, 4])', '✓ product([10, 2, 50])', '✓ product([])', '🎉 Congratulations! You have successfully solved problem product!!']}, {'problem': 'digit-count', 'status': 'pass', 'log': ['✓ digit_count(1231, 1)', '✓ digit_count(1231, 3)', '✓ digit_count(1231, 9)', '✓ digit_count(2**1000, 0)', '🎉 Congratulations! You have successfully solved problem digit-count!!']}, {'problem': 'print-header', 'status': 'pass', 'log': ['✓ python header.py python', '✓ python header.py Python-Foundation-Course', '🎉 Congratulations! You have successfully solved problem print-header!!']}, {'problem': 'text-in-a-box', 'status': 'pass', 'log': ['✓ python box.py python', '✓ python box.py Joy', '✓ python box

In [2]:

path = "archive/assignment-01/abhishek.ipynb"
nb = nbformat.read(open(path), as_version=4)

nb['cells'].insert(0, precode)
nb['cells'].append(nbformat.v4.new_code_cell(code))

In [3]:
nb['cells'][-1]

{'cell_type': 'code',
 'execution_count': None,
 'id': '314ce6bb-7c99-4718-b341-5ce7974dcd2f',
 'metadata': {},
 'outputs': [],
 'source': ''}

In [4]:
code = "print(VERIFY_PROBLEM_GRADES)"

In [5]:
nb['cells'].append(nbformat.v4.new_code_cell(code))

In [6]:
p = ExecutePreprocessor(timeout=600, kernel_name='python3', allow_errors=True)

In [7]:
nb2 = p.preprocess(nb, {'metadata': {'path': 'tmp/'}})

In [8]:
with open("a.ipynb", "w") as f:
    nbformat.write(nb2[0], f)

In [8]:
# nb2

In [17]:
# nb2[0]['cells']

In [7]:
import os

In [14]:
os.getenv("VERIFY_PROBLEM_MODE")

In [12]:
os.environ.get("VERIFY_PROBLEM_MODE")

In [11]:
os.putenv("VERIFY_PROBLEM_MODE", "grading")

In [15]:
!python -c "import os; os.getenv('VERIFY_PROBLEM_MODE')"

## Sigma

In [5]:
import sigma

In [6]:
sigma.__file__

'/home/jupyter-vikrant/arcesium-python-2024/sigma/__init__.py'

In [7]:
from sigma import Problem, config
import nbformat as nbf
import yaml

In [8]:
config.PROBLEM_REPOSITORY_PATH = "/opt/training/problems"

In [9]:
nb = nbf.v4.new_notebook()

In [10]:
nb

{'nbformat': 4, 'nbformat_minor': 5, 'metadata': {}, 'cells': []}

In [11]:
def add_markdown_cell(text):
    cell = nbf.v4.new_markdown_cell(text)
    nb['cells'].append(cell)

def add_code_cell(code):
    cell = nbf.v4.new_code_cell(code)
    nb['cells'].append(cell)
    
def add_title(title):
    add_markdown_cell(f"# {title}")
    
def add_problem(index, p):
    add_markdown_cell(f"## Problem {index}: {p.title}")

    add_markdown_cell(
        p.get_description() 
        + "\n\n"
        + "You can verify your solution using:<br>\n"
        + f"`%verify_problem {p.name}`")

    if p.problem_type == "script":
        add_code_cell(f"%%file {p.script_name}\n# your code here")
    else:
        code = p.get_initial_code() or "# your code here\n\n\n\n"
        add_code_cell(code)
    
    add_code_cell("")
    add_code_cell("")
    add_code_cell(f"%verify_problem {p.name}")

def create_notebook(title, problems):
    global nb
    nb = nbf.v4.new_notebook()
    add_title(title)
    for i, p in enumerate(problems, start=1):
        add_problem(i, p)

In [12]:
assignments = {a['id']: a for a in yaml.safe_load(open("assignments.yml"))}

In [13]:
def get_problems(assignment: int):
    names = assignments[assignment]['problems']
    print("get_problems", assignment, names)
    def find(name):
        print("find", name)
        return Problem.find(name)
    return [find(name) for name in names]

In [14]:
problems = get_problems(3)

get_problems 3 ['combined-daily-time-series', 'dict-client', 'frankfurter']
find combined-daily-time-series
find dict-client
find frankfurter


In [15]:
create_notebook("Assignment 3", problems)
nbf.write(nb, "assignment-03.ipynb")

## Assignment Solutions

In [22]:
TEMPLATE = """
# {{title}}

Solutions to {{title}}.

{% for problem in problems %}
## {{ problem.title }}

{{problem.get_description()}}

### Solution {.unnumbered}

```python
{{ get_solution(problem) }}
```

{% set discussion = get_discussion(problem) %}

{% if discussion %}
### Discussion {.unnumbered }

{{ discussion }}

{% endif %}

{% endfor %}
"""

In [23]:
from jinja2 import Template

In [24]:
t = Template(TEMPLATE)

In [25]:
def get_solution(problem):
    solution_file = problem.metadata['files']['solution'][0]
    path = Path(config.PROBLEM_REPOSITORY_PATH) / problem.name / solution_file
    return path.read_text()

def get_discussion(problem):
    path = Path(config.PROBLEM_REPOSITORY_PATH) / problem.name / "solution" / "discussion.md"
    return path.read_text() if path.exists() else ""

def render_solutions(assignment: int):
    title = f"Assignment {assignment:02d}"
    problems = get_problems(assignment)
    qmd = t.render(title=title, problems=problems, 
                   get_solution=get_solution,
                   get_discussion=get_discussion)

    path = f"../book/assignments/assignment-{assignment:02d}.qmd"
    with open(path, "w") as f:
        f.write(qmd)
    print("Generated", path)

In [26]:
render_solutions(1)

NameError: name 'get_problems' is not defined

In [182]:
render_solutions(2)

get_problems 2 ['despace', 'sort-words', 'minimum3', 'longest-line', 'line-with-most-words', 'list-files', 'sumfile', 'grep', 'cumulative-sum', 'count-increased']
find despace
find sort-words
find minimum3
find longest-line
find line-with-most-words
find list-files
find sumfile
find grep
find cumulative-sum
find count-increased
Generated ../book/assignments/assignment-02.qmd


In [16]:
render_solutions(3)

get_problems 3 ['vector-add', 'group', 'longest-argument', 'paste-cmd', 'center-align']
find vector-add
find group
find longest-argument
find paste-cmd
find center-align
Generated ../book/assignments/assignment-03.qmd


In [46]:
render_solutions(4)

get_problems 4 ['reverse-words', 'seq', 'tail', 'skip-cmd', 'split']
find reverse-words
find seq
find tail
find skip-cmd
find split
Generated ../book/assignments/assignment-04.qmd


In [44]:
render_solutions(5)

get_problems 5 ['unique', 'flatten', 'extcount', 'sales-by-day', 'uniq-cmd']
find unique
find flatten
find extcount
find sales-by-day
find uniq-cmd
Generated ../book/assignments/assignment-05.qmd


In [45]:
render_solutions(6)

get_problems 6 ['reverse-lines', 'generate-password', 'weekly-average', 'sort-cmd', 'hosts-writer']
find reverse-lines
find generate-password
find weekly-average
find sort-cmd
find hosts-writer
Generated ../book/assignments/assignment-06.qmd


In [35]:
render_solutions(7)

get_problems 7 ['nominatim', 'dict-client', 'frankfurter', 'pypi-releases', 'gist-lib']
find nominatim
find dict-client
find frankfurter
find pypi-releases
find gist-lib
Generated ../book/assignments/assignment-07.qmd


In [36]:
render_solutions(8)

get_problems 8 ['slug', 'df', 'dig', 'run-length-encoding', 'two-column-layout']
find slug
find df
find dig
find run-length-encoding
find two-column-layout
Generated ../book/assignments/assignment-08.qmd
